### Nesta 2° parte da disciplina será utilizado o mesmo projeto da 1° unidade, entretanto iremos realizar:
- uma limpeza, tratamento e análise dos dados utilizando o PySpark;
- Criação de um relatório com os insights obtidos.

### Ferramentas utilizadas:
- PySpark
- gdown
- numpy
- pandas
- seaborn

In [7]:
# 1° ato: carregamento dos dados
import gdown
import pandas as pd
from pyspark.sql import SparkSession

# URL do dados Recife
url = "http://dados.recife.pe.gov.br/dataset/e2bd8f0b-1d62-4323-8159-8ebd6ed9eb4a/resource/77c885c4-76ca-45eb-9209-06c5d217122d/download/licenciamento_urbanistico.csv"

# Baixando o arquivo
output = 'licenciamento_urbanistico.csv'
gdown.download(url, output, quiet=False)

# Verificando o arquivo com pandas
data = pd.read_csv(output, sep=';')
print(data.head())

# Inicializando o SparkSession
spark = SparkSession.builder.appName('licenciamento-urbanistico').getOrCreate()

# Ajustando a configuração para o plano de string
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

# Lendo o arquivo CSV baixado
df = spark.read.csv(output, header=True, sep=';', inferSchema=True)
df.printSchema()

# Mostrar as primeiras linhas do DataFrame
df.show(5)



Downloading...
From: http://dados.recife.pe.gov.br/dataset/e2bd8f0b-1d62-4323-8159-8ebd6ed9eb4a/resource/77c885c4-76ca-45eb-9209-06c5d217122d/download/licenciamento_urbanistico.csv
To: /home/victoria/PycharmProjects/Big-Data/Trabalho/Parte2/licenciamento_urbanistico.csv
41.0MB [01:37, 421kB/s] 


  ﻿areatotalconstruida                       assunto            bairro  \
0                  100  APROVACAO DE PROJETO INICIAL            AREIAS   
1                  NaN                  OBRA DE ARTE  BRASILIA TEIMOSA   
2                  NaN                  OBRA DE ARTE  BRASILIA TEIMOSA   
3                   50  APROVACAO DE PROJETO INICIAL            AREIAS   
4                  100  APROVACAO DE PROJETO INICIAL            AREIAS   

  categoria_empreendimento          cnpj data_conclusao data_emissao_licenca  \
0                     Obra  2.088921e+13     2017-05-29           2017-05-29   
1                     Obra  2.088921e+13     2017-05-30                  NaN   
2                     Obra  2.088921e+13     2017-06-01                  NaN   
3                     Obra  2.088921e+13     2017-07-21                  NaN   
4                     Obra  2.088921e+13     2017-07-21                  NaN   

  data_entrada data_pagamento data_validade_licenca  ...  \
0   2017-05-29

root
 |-- ﻿areatotalconstruida: string (nullable = true)
 |-- assunto: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- categoria_empreendimento: string (nullable = true)
 |-- cnpj: long (nullable = true)
 |-- data_conclusao: date (nullable = true)
 |-- data_emissao_licenca: date (nullable = true)
 |-- data_entrada: date (nullable = true)
 |-- data_pagamento: date (nullable = true)
 |-- data_validade_licenca: date (nullable = true)
 |-- dsqfl: string (nullable = true)
 |-- empreendimento_de_impacto: string (nullable = true)
 |-- endereco_empreendimento: string (nullable = true)
 |-- inscricao_mercantil: integer (nullable = true)
 |-- licenciamento_ambiental: string (nullable = true)
 |-- licenciamento_sanitario: string (nullable = true)
 |-- licenciamento_urbanistico: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- razao_social_mercantil: string (nullable = true)
 |-- num_licenca: long (nullable = true)
 |-- num_processo: long (nullable = tr

In [8]:
pd.DataFrame(df.take(5), columns=df.columns)

,﻿areatotalconstruida,assunto,bairro,categoria_empreendimento,cnpj,data_conclusao,data_emissao_licenca,data_entrada,data_pagamento,data_validade_licenca,...,processo_automatizado,situacao_processo,tipo_mercantil,tipo_processo,tipo_proc_licenciamento,uso_imovel,valor_taxa,valor_pago,latitude,longitude
0,100,APROVACAO DE PROJETO INICIAL,AREIAS,Obra,20889207000176,2017-05-29,2017-05-29,2017-05-29,None,2018-05-29,...,Sim,DEFERIDO,MEI,None,Urbanístico,None,None,None,-8.089069,-34.934006
1,None,OBRA DE ARTE,BRASILIA TEIMOSA,Obra,20889207000176,2017-05-30,None,2017-05-30,None,None,...,Sim,INDEFERIDO,MEI,None,Urbanístico,None,None,None,-8.089069,-34.934006
2,None,OBRA DE ARTE,BRASILIA TEIMOSA,Obra,20889207000176,2017-06-01,None,2017-05-30,None,None,...,Sim,INDEFERIDO,MEI,None,Urbanístico,None,None,None,-8.089069,-34.934006
3,50,APROVACAO DE PROJETO INICIAL,AREIAS,Obra,20889207000176,2017-07-21,None,2017-05-29,None,None,...,Sim,CANCELADO,MEI,None,Urbanístico,None,None,None,-8.089069,-34.934006
4,100,APROVACAO DE PROJETO INICIAL,AREIAS,Obra,20889207000176,2017-07-21,None,2017-05-29,None,None,...,Sim,CANCELADO,MEI,None,Urbanístico,None,None,None,-8.089069,-34.934006


In [9]:
df.groupby('situacao_processo').count().toPandas()

,situacao_processo,count
0,INDEFERIDO,32965
1,DEFERIDO,28601
2,CANCELADO,11074
3,EM TRAMITAÇÃO,522
4,SOLICITADA,1029
5,EM EXIGÊNCIA,776


In [10]:
df.groupby('licenciamento_urbanistico').count().toPandas()

,licenciamento_urbanistico,count
0,Licenciamento(s) solicitado(s) - 8083232623 - ...,11
1,Licenciamento vigente - ALVARA DE CONSTRUCAO I...,27
2,Licenciamento vigente - ALVARA CONSTRUCAO/REFORMA,310
3,Licenciamento vigente - ALVARA DE CONSTRUCAO I...,827
4,Licenciamento expirado - ALVARA DE CONSTRUCAO/...,101
...,...,...
802,Licenciamento(s) solicitado(s) - 8001039024 - ...,1
803,Licenciamento(s) solicitado(s) - 8080503923 - ...,1
804,Licenciamento(s) solicitado(s) - 8073758523 - ...,2
805,Licenciamento(s) solicitado(s) - 8079035523 - ...,1
